<a href="https://colab.research.google.com/github/sag005/spotify-playlist-recommendation/blob/main/Popularity_Based_Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle as pkl
from collections import defaultdict
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
import math
import warnings

In [ ]:
tracks_data = pd.read_pickle('track_data.pkl')
playlists_data = pd.read_pickle('playlist_data.pkl')

In [ ]:
artist_tracks_dict = {}
with open('artist_track_mapping.pkl', 'rb') as handle:
    artist_tracks_dict = pkl.load(handle)

In [ ]:
track_artist_mapping = {}
with open('track_artist_mapping.pkl', 'rb') as handle:
    track_artist_mapping = pkl.load(handle) 

In [ ]:
track_frequency = {}
with open('track_frequency.pkl', 'rb') as handle:
    track_frequency = pkl.load(handle)

In [ ]:
artist_tracks_count_dict = {}
with open('artist_track_count.pkl', 'rb') as handle:
    artist_tracks_count_dict = pkl.load(handle)

In [ ]:
def getArtistsForTracks(tracks):
  return [track_artist_mapping[track] for track in tracks]

In [ ]:
def R_precision(predicted_tracks, with_held_tracks):
  # print(len(set(with_held_tracks)))
  intersection = len(set(predicted_tracks).intersection(set(with_held_tracks)))
  return intersection / len(set(with_held_tracks))

In [43]:
def NDCG(truth, pred, K):
  if len(pred) > 0:
    rel_truth = np.zeros((1, len(pred)))
    rel_pred = np.zeros((1, len(pred)))
    count_t = 0
    for t in truth:
      if t in pred:
        idx = np.where(np.array(pred) == t)[0]
        # print(idx)
        rel_pred[0, idx[0]]=1
        rel_truth[0, count_t]=1
      count_t+=1
    return ndcg_score(rel_truth, rel_pred, k=K)
  else:
    return 0

In [ ]:
def getMostPopularSongs(n, seed_playlist):
  songs_with_freq_list = []
  for song in track_frequency:
    songs_with_freq_list.append((track_frequency[song], song))
  songs_with_freq_list.sort(reverse=True)
  res = [pair[1] for pair in songs_with_freq_list if pair[1] not in seed_playlist]
  return res[:n]

In [ ]:
getArtistsForTracks(getMostPopularSongs(100, ['spotify:track:3a1lNhkSLSkpJE4MSHpDu9']))

In [ ]:
validation_data = pd.read_pickle('Validation_dataset.pkl')

In [ ]:
def runPopularTrackModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    for idx, row in validation_data.iterrows():
      res = getMostPopularSongs(topK, row['tracks'])
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    # print('K=',topK, ' ,NDCG(Artist)=',NDCG(row['with_held_artists'], artists_of_pred))
    print('=====================================================')

In [ ]:
runPopularTrackModel([10, 15, 20, 25, 30,35, 50])

K= 10  ,RP(Artist)= 0.03917375283446708
K= 10  ,RP(Track)= 0.0
K= 10  ,NDCG(Track)= 0.0
K= 15  ,RP(Artist)= 0.07211451247165528
K= 15  ,RP(Track)= 0.0
K= 15  ,NDCG(Track)= 0.0
K= 20  ,RP(Artist)= 0.08119047619047612
K= 20  ,RP(Track)= 0.0
K= 20  ,NDCG(Track)= 0.0
K= 25  ,RP(Artist)= 0.09361536281179131
K= 25  ,RP(Track)= 0.0
K= 25  ,NDCG(Track)= 0.0
K= 30  ,RP(Artist)= 0.11507369614512462
K= 30  ,RP(Track)= 0.0
K= 30  ,NDCG(Track)= 0.0
K= 35  ,RP(Artist)= 0.1206533446712017
K= 35  ,RP(Track)= 0.0
K= 35  ,NDCG(Track)= 0.0
K= 50  ,RP(Artist)= 0.14615787981859402
K= 50  ,RP(Track)= 0.0
K= 50  ,NDCG(Track)= 0.0


### Recommend Songs of only those Artists corresponding to Seed playlist

In [ ]:
def getMostPopularSongsForArtist(seedTracks, artist_name, n, tracks_to_choose_from=None):
  if tracks_to_choose_from is None:
    artist_songs = artist_tracks_dict[artist_name]
    return random.sample(artist_songs, min(n, len(artist_songs)))
  else:
    track_list = tracks_to_choose_from[artist_name]
    return random.sample(track_list, min(n, len(track_list)))
  # artist_popular_songs = []
  # for song in artist_songs:
    # artist_popular_songs.append((track_frequency[song], song))
    # artist_popular_songs.sort(reverse=True)
    # filtered = [pair[1] for pair in artist_popular_songs if pair[1] not in seedTracks]
  # return filtered[:n]



In [ ]:
def recommendPopularSongsBasedOnArtists(seedTracks, artists, n, tracks_to_choose_from=None):
  total_tracks = 0
  for artist in artists:
    total_tracks += artist_tracks_count_dict[artist]
  results = []
  for artist in artists:
    count = int(math.ceil((n * artist_tracks_count_dict[artist]) / total_tracks))
    results.extend(getMostPopularSongsForArtist(seedTracks, artist, count, tracks_to_choose_from))
  return results[:n]

In [ ]:
def runPopularArtistTrackModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    for idx, row in validation_data.iterrows():
      artists = getArtistsForTracks(row['tracks'])
      res = recommendPopularSongsBasedOnArtists(row['tracks'], artists, topK, None)
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('=====================================================')

In [ ]:
runPopularArtistTrackModel([10, 13, 15, 18, 20, 23, 25])

K= 10  ,RP(Artist)= 0.20582766439909317
K= 10  ,RP(Track)= 0.021904761904761896
K= 10  ,NDCG(Track)= 0.060324276319645036
K= 13  ,RP(Artist)= 0.2688633786848073
K= 13  ,RP(Track)= 0.03329365079365077
K= 13  ,NDCG(Track)= 0.10126521894759051
K= 15  ,RP(Artist)= 0.3113704648526079
K= 15  ,RP(Track)= 0.04757936507936503
K= 15  ,NDCG(Track)= 0.1276837263700039
K= 18  ,RP(Artist)= 0.36096513605442165
K= 18  ,RP(Track)= 0.056865079365079306
K= 18  ,NDCG(Track)= 0.12883176559290083
K= 20  ,RP(Artist)= 0.3876218820861679
K= 20  ,RP(Track)= 0.059761904761904724
K= 20  ,NDCG(Track)= 0.1235657500727709
K= 23  ,RP(Artist)= 0.43256660997732416
K= 23  ,RP(Track)= 0.07738095238095247
K= 23  ,NDCG(Track)= 0.15223034029362742
K= 25  ,RP(Artist)= 0.45637188208616764
K= 25  ,RP(Track)= 0.08130952380952385
K= 25  ,NDCG(Track)= 0.15593943396180382


Higher artist R@K shows users have favourite artists and its common to have majority of playlist composed of their tracks.

In [ ]:
runPopularArtistTrackModel([10, 20, 30, 40, 50, 60, 70])

K= 10  ,RP(Artist)= 0.20582766439909317
K= 10  ,RP(Track)= 0.026547619047619035
K= 10  ,NDCG(Track)= 0.08894812439814553
K= 20  ,RP(Artist)= 0.3876218820861679
K= 20  ,RP(Track)= 0.06376984126984127
K= 20  ,NDCG(Track)= 0.14513118094492053
K= 30  ,RP(Artist)= 0.5199503968253968
K= 30  ,RP(Track)= 0.10285714285714292
K= 30  ,NDCG(Track)= 0.17523031371260128
K= 40  ,RP(Artist)= 0.5976204648526077
K= 40  ,RP(Track)= 0.12888888888888908
K= 40  ,NDCG(Track)= 0.20068097021574302
K= 50  ,RP(Artist)= 0.6688619614512475
K= 50  ,RP(Track)= 0.15654761904761924
K= 50  ,NDCG(Track)= 0.2074167065632705
K= 60  ,RP(Artist)= 0.7053472222222231
K= 60  ,RP(Track)= 0.1983333333333336
K= 60  ,NDCG(Track)= 0.22943842732628913
K= 70  ,RP(Artist)= 0.7326218820861684
K= 70  ,RP(Track)= 0.21670634920634962
K= 70  ,NDCG(Track)= 0.23867005459136342


### Cosine Similarity of Playlist Title Word Embeddings - Find the similarity between query Playlist title and Trainig Playlist Titles(acting as genere)

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/glove_emb.pb (1).zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
words_embeddings_dict = defaultdict()
with open('glove_emb.pb', 'rb') as handle:
    words_embeddings_dict = pkl.load(handle)

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
playlist_names = playlists_data['name']
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def getPlaylistTitleEmbedding(name):
  # print(name)
  embeddies = np.zeros((1,300), dtype=float)
  for w in name:
    # print(w)
    if w.lower() in words_embeddings_dict:
      embeddies += np.array(words_embeddings_dict[w.lower()]).reshape((1,300))
  return embeddies

In [ ]:
def tracksInPlaylist(p):
  # print(p, len(playlists_data.loc[playlists_data['name']==p,'tracks']))
  return list(playlists_data.loc[playlists_data['name']==p,'tracks'].iloc[0])

In [ ]:
def getMostSimilarPlaylist(query_playlist):
  sim_matrix = []
  # best_match = ''
  name1 = [w.lower() for w in query_playlist.strip().split(' ') if not w.lower() in stop_words]
  embedding1 = getPlaylistTitleEmbedding(name1)
  for n2 in playlist_names.to_list():
    # print(n2)
    name2 = [w.lower() for w in n2.strip().split(' ') if not w.lower() in stop_words]
    embedding2 = getPlaylistTitleEmbedding(name2)
    sim = 1 - distance.cosine(embedding2, embedding1)
    sim_matrix.append((sim, n2))
    # print(sim, query_playlist, n2)
    # if(sim > max_sim):
      # max_sim = sim
      # best_match = n2
  return sim_matrix

In [44]:
def getRandomTopNFromBestMatchTitles(query_playlist, n, playlist_cnt):
  sim_matrix = getMostSimilarPlaylist(query_playlist)
  # print(sim_matrix)
  sim_matrix = [pair for pair in sim_matrix if not np.isnan(pair[0])]
  sim_matrix.sort(reverse=True)
  # print(sim_matrix)
  total_sim = sum([pair[0] for pair in sim_matrix[:playlist_cnt]])
  result = []
  for p in sim_matrix[:playlist_cnt]:
    tracks = tracksInPlaylist(p[1])
    weight = int(math.ceil(n * p[0] / total_sim))
    result.extend(random.sample(tracks, min(weight, len(tracks))))
  return result[:n]


In [49]:
def runWordEmbeddingsModel(topKgrid, playlist_cnt):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    for idx, row in validation_data.iterrows():
      res = getRandomTopNFromBestMatchTitles(row['name'], topK, playlist_cnt)
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    # print(ndcg_tracks)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('=====================================================')

In [46]:
warnings.filterwarnings('ignore')
runWordEmbeddingsModel([10])

K= 10  ,RP(Artist)= 0.04767290249433103
K= 10  ,RP(Track)= 0.01579365079365079
K= 10  ,NDCG(Track)= 0.058864235131331655


In [47]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70], 1)

K= 10  ,RP(Artist)= 0.04443452380952378
K= 10  ,RP(Track)= 0.016150793650793648
K= 10  ,NDCG(Track)= 0.04749126542889242
K= 20  ,RP(Artist)= 0.07866780045351472
K= 20  ,RP(Track)= 0.035873015873015855
K= 20  ,NDCG(Track)= 0.07523550368207549
K= 30  ,RP(Artist)= 0.08769416099773243
K= 30  ,RP(Track)= 0.04246031746031743
K= 30  ,NDCG(Track)= 0.07338711085350866
K= 40  ,RP(Artist)= 0.09688492063492064
K= 40  ,RP(Track)= 0.047460317460317425
K= 40  ,NDCG(Track)= 0.07576976063802042
K= 50  ,RP(Artist)= 0.09986536281179138
K= 50  ,RP(Track)= 0.049603174603174566
K= 50  ,NDCG(Track)= 0.07773292022646042
K= 60  ,RP(Artist)= 0.10112953514739229
K= 60  ,RP(Track)= 0.049603174603174566
K= 60  ,NDCG(Track)= 0.07546367620933282
K= 70  ,RP(Artist)= 0.10112953514739229
K= 70  ,RP(Track)= 0.049603174603174566
K= 70  ,NDCG(Track)= 0.07813899661041797


## with top 4 matches

In [50]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100], 4)

K= 10  ,RP(Artist)= 0.03841695011337867
K= 10  ,RP(Track)= 0.009761904761904765
K= 10  ,NDCG(Track)= 0.03897727857504507
K= 20  ,RP(Artist)= 0.06795918367346934
K= 20  ,RP(Track)= 0.022738095238095227
K= 20  ,NDCG(Track)= 0.051176635791984955
K= 30  ,RP(Artist)= 0.09312216553287982
K= 30  ,RP(Track)= 0.02841269841269839
K= 30  ,NDCG(Track)= 0.06499735484896615
K= 40  ,RP(Artist)= 0.10375425170068023
K= 40  ,RP(Track)= 0.03670634920634919
K= 40  ,NDCG(Track)= 0.056207923509556314
K= 50  ,RP(Artist)= 0.1197619047619048
K= 50  ,RP(Track)= 0.044999999999999964
K= 50  ,NDCG(Track)= 0.06902460481169037
K= 60  ,RP(Artist)= 0.12563066893424044
K= 60  ,RP(Track)= 0.046388888888888855
K= 60  ,NDCG(Track)= 0.07121741319691804
K= 70  ,RP(Artist)= 0.14307539682539697
K= 70  ,RP(Track)= 0.05432539682539678
K= 70  ,NDCG(Track)= 0.06877270708189817
K= 100  ,RP(Artist)= 0.16026077097505678
K= 100  ,RP(Track)= 0.06543650793650795
K= 100  ,NDCG(Track)= 0.07950037360006751


## with top 3 matches

In [51]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100], 3)

K= 10  ,RP(Artist)= 0.043782596371882046
K= 10  ,RP(Track)= 0.015436507936507936
K= 10  ,NDCG(Track)= 0.05305519551256277
K= 20  ,RP(Artist)= 0.07321286848072561
K= 20  ,RP(Track)= 0.020555555555555553
K= 20  ,NDCG(Track)= 0.04636974591746453
K= 30  ,RP(Artist)= 0.08793792517006804
K= 30  ,RP(Track)= 0.034166666666666644
K= 30  ,NDCG(Track)= 0.07378663211829434
K= 40  ,RP(Artist)= 0.10157171201814055
K= 40  ,RP(Track)= 0.04138888888888886
K= 40  ,NDCG(Track)= 0.06919859703932449
K= 50  ,RP(Artist)= 0.1174263038548753
K= 50  ,RP(Track)= 0.04571428571428568
K= 50  ,NDCG(Track)= 0.06713448114243037
K= 60  ,RP(Artist)= 0.1256604308390023
K= 60  ,RP(Track)= 0.04785714285714282
K= 60  ,NDCG(Track)= 0.06810164735541765
K= 70  ,RP(Artist)= 0.13467120181405903
K= 70  ,RP(Track)= 0.053928571428571374
K= 70  ,NDCG(Track)= 0.07881179922459765
K= 100  ,RP(Artist)= 0.1494075963718822
K= 100  ,RP(Track)= 0.06119047619047616
K= 100  ,NDCG(Track)= 0.08192594730377994


## with top 2 matches

In [52]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100], 2)

K= 10  ,RP(Artist)= 0.043443877551020374
K= 10  ,RP(Track)= 0.013293650793650795
K= 10  ,NDCG(Track)= 0.03585348832977778
K= 20  ,RP(Artist)= 0.07197137188208615
K= 20  ,RP(Track)= 0.02738095238095237
K= 20  ,NDCG(Track)= 0.05819961734369706
K= 30  ,RP(Artist)= 0.09396258503401358
K= 30  ,RP(Track)= 0.035277777777777755
K= 30  ,NDCG(Track)= 0.06633111859675886
K= 40  ,RP(Artist)= 0.10441326530612242
K= 40  ,RP(Track)= 0.04420634920634917
K= 40  ,NDCG(Track)= 0.06861075877098596
K= 50  ,RP(Artist)= 0.10850481859410425
K= 50  ,RP(Track)= 0.04753968253968251
K= 50  ,NDCG(Track)= 0.07564460980086281
K= 60  ,RP(Artist)= 0.1221428571428572
K= 60  ,RP(Track)= 0.05361111111111106
K= 60  ,NDCG(Track)= 0.07001987201286032
K= 70  ,RP(Artist)= 0.13125850340136064
K= 70  ,RP(Track)= 0.055753968253968195
K= 70  ,NDCG(Track)= 0.07827652638769236
K= 100  ,RP(Artist)= 0.13634212018140598
K= 100  ,RP(Track)= 0.05896825396825393
K= 100  ,NDCG(Track)= 0.07488448449308312


In [53]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100], 10)

K= 10  ,RP(Artist)= 0.0381391723356009
K= 10  ,RP(Track)= 0.00642857142857143
K= 10  ,NDCG(Track)= 0.03153651242888419
K= 20  ,RP(Artist)= 0.0657738095238095
K= 20  ,RP(Track)= 0.016230158730158727
K= 20  ,NDCG(Track)= 0.042408790842219914
K= 30  ,RP(Artist)= 0.0870379818594104
K= 30  ,RP(Track)= 0.023253968253968253
K= 30  ,NDCG(Track)= 0.047655668496997476
K= 40  ,RP(Artist)= 0.11308390022675736
K= 40  ,RP(Track)= 0.030952380952380926
K= 40  ,NDCG(Track)= 0.06573097700932008
K= 50  ,RP(Artist)= 0.13639314058956922
K= 50  ,RP(Track)= 0.037103174603174575
K= 50  ,NDCG(Track)= 0.07099436626139322
K= 60  ,RP(Artist)= 0.1411323696145126
K= 60  ,RP(Track)= 0.045238095238095195
K= 60  ,NDCG(Track)= 0.08523759253985323
K= 70  ,RP(Artist)= 0.14659297052154202
K= 70  ,RP(Track)= 0.05059523809523804
K= 70  ,NDCG(Track)= 0.08352049241031158
K= 100  ,RP(Artist)= 0.179029195011338
K= 100  ,RP(Track)= 0.06539682539682541
K= 100  ,NDCG(Track)= 0.08470116733891354


##Combine Word Embeddings with Artist popularity

In [54]:
def getTrackDictForArtistWithinPlayists(sim_matrix):
  all_tracks= []
  for p in sim_matrix:
    all_tracks.extend(tracksInPlaylist(p[1]))
  artist_track_dict = defaultdict(list)
  for track in all_tracks:
    artist_track_dict[track_artist_mapping[track]].append(track)
  return artist_track_dict

In [55]:
def runPopularArtistPlusWordEmbeddingModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    for idx, row in validation_data.iterrows():
      sim_matrix = getMostSimilarPlaylist(row['name'])
      sim_matrix = [pair for pair in sim_matrix if not np.isnan(pair[0])]
      sim_matrix.sort(reverse=True)
      tracks_to_choose_from = getTrackDictForArtistWithinPlayists(sim_matrix[:20])
      res = recommendPopularSongsBasedOnArtists(row['tracks'], list(tracks_to_choose_from.keys()), topK, tracks_to_choose_from)
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('=====================================================')

In [56]:
runPopularArtistPlusWordEmbeddingModel([10, 20, 30, 40, 50, 60, 70, 100])

K= 10  ,RP(Artist)= 0.06216128117913829
K= 10  ,RP(Track)= 0.01301587301587302
K= 10  ,NDCG(Track)= 0.048315354148027116
K= 20  ,RP(Artist)= 0.09726899092970523
K= 20  ,RP(Track)= 0.025198412698412685
K= 20  ,NDCG(Track)= 0.058512025778202456
K= 30  ,RP(Artist)= 0.12450538548752842
K= 30  ,RP(Track)= 0.029087301587301568
K= 30  ,NDCG(Track)= 0.06067194781868334
K= 40  ,RP(Artist)= 0.14595096371882102
K= 40  ,RP(Track)= 0.03734126984126981
K= 40  ,NDCG(Track)= 0.07399425398276945
K= 50  ,RP(Artist)= 0.16419217687074844
K= 50  ,RP(Track)= 0.04345238095238091
K= 50  ,NDCG(Track)= 0.07631631634538912
K= 60  ,RP(Artist)= 0.18302295918367362
K= 60  ,RP(Track)= 0.04638888888888884
K= 60  ,NDCG(Track)= 0.07543499597435674
K= 70  ,RP(Artist)= 0.19517006802721104
K= 70  ,RP(Track)= 0.048809523809523754
K= 70  ,NDCG(Track)= 0.07278133910441542
K= 100  ,RP(Artist)= 0.23253259637188245
K= 100  ,RP(Track)= 0.06246031746031743
K= 100  ,NDCG(Track)= 0.08485086070108266


##With 20 similar playlists

In [ ]:
runPopularArtistPlusWordEmbeddingModel([10, 20, 30, 40, 50, 60, 70])

Word embeddings are not giving ggod results not even with same artists as of seed list. Which means not all songs are related to the title of the playlist. The songs are random and do not describe the overall genere of songs. 